In [1]:
!pip uninstall opencv-python -y
# downgrade OpenCV a bit since some none-free features are not avilable
!pip install opencv-contrib-python==3.4.2.17

Uninstalling opencv-python-4.1.2.30:
  Successfully uninstalled opencv-python-4.1.2.30
     |████████████████████████████████| 30.6MB 107kB/s 
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


#Join 4 frames 
e.g frame 1, frame 2, frame 3 and 4. by calculating the homography to frame 1. 

##Get data

In [2]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# List .txt files in the root.
#
# Search query reference:
# https://developers.google.com/drive/v2/web/search-parameters
listed = drive.ListFile({'q': "title contains 'flow_files_video06.zip'"}).GetList()
for flow_file in listed:
  print('title {}, id {}'.format(flow_file['title'], flow_file['id']))

listed = drive.ListFile({'q': "title contains 'cropped_video06_images.zip'"}).GetList()
for video_file in listed:
  print('title {}, id {}'.format(video_file['title'], video_file['id']))

listed = drive.ListFile({'q': "title contains 'masks.zip'"}).GetList()
for mask_file in listed:
  print('title {}, id {}'.format(mask_file['title'], mask_file['id']))

# listed = drive.ListFile({'q': "title contains 'video03.zip'"}).GetList()
# for file in listed:
#   print('title {}, id {}'.format(file['title'], file['id']))

title flow_files_video06.zip, id 1-hdToh-ci9TqPqH4dnSKZ145kgHDGB72
title cropped_video06_images.zip, id 1pD8qrbFxlOPlVPQr-ZKm7s_lhMOc-xnC
title masks.zip, id 1lBgLfSBD_8d0PxBSEI9uP7W3o-8bnZzW


In [3]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


file_id = flow_file['id']

fname = os.path.join('/content', 'flow_files.zip')
print('downloading to {}'.format(fname))
f_ = drive.CreateFile({'id': file_id})
f_.GetContentFile(fname)


file_id = video_file['id']

fname = os.path.join('/content', 'video.zip')
print('downloading to {}'.format(fname))
f_ = drive.CreateFile({'id': file_id})
f_.GetContentFile(fname)


file_id = mask_file['id']

fname = os.path.join('/content', 'masks.zip')
print('downloading to {}'.format(fname))
f_ = drive.CreateFile({'id': file_id})
f_.GetContentFile(fname)

downloading to /content/flow_files.zip
downloading to /content/video.zip
downloading to /content/masks.zip


In [4]:
!unzip flow_files.zip
!unzip masks.zip
!unzip video.zip

Archive:  flow_files.zip
   creating: flow_files_video06/
  inflating: flow_files_video06/000000.flo  
  inflating: flow_files_video06/000001.flo  
  inflating: flow_files_video06/000002.flo  
  inflating: flow_files_video06/000003.flo  
  inflating: flow_files_video06/000004.flo  
  inflating: flow_files_video06/000005.flo  
  inflating: flow_files_video06/000006.flo  
  inflating: flow_files_video06/000007.flo  
  inflating: flow_files_video06/000008.flo  
  inflating: flow_files_video06/000009.flo  
  inflating: flow_files_video06/000010.flo  
  inflating: flow_files_video06/000011.flo  
  inflating: flow_files_video06/000012.flo  
  inflating: flow_files_video06/000013.flo  
  inflating: flow_files_video06/000014.flo  
  inflating: flow_files_video06/000015.flo  
  inflating: flow_files_video06/000016.flo  
  inflating: flow_files_video06/000017.flo  
  inflating: flow_files_video06/000018.flo  
  inflating: flow_files_video06/000019.flo  
  inflating: flow_files_video06/000020.flo

In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import imageio
cv2.ocl.setUseOpenCL(False)
import sys
from PIL import Image
from pathlib import Path

In [6]:
from google.colab.patches import cv2_imshow

In [7]:
transformation = "Affine"
video_name = "video06"

In [8]:
# !pip install flowiz -U

In [9]:
# import flowiz as fz

In [10]:
def inputAndVisualizeStitchPair(srcImgPath, destImgPath, showImages=True): 
  '''
    Inputs: srcImgPath, destImgPath 
    srcImgPath : path to the image been transformed, usually the new frame seen ->str
    destImgPath : path to the image that acts as destination(src image is trasformed to it), usually the old frame->str
    Output: [srcImg, destImg] -> list
    srcImg: 256x256 representation of srcImage ->ndarray
    destImg: 256x256 representation of destImage ->ndarray
  '''
  srcImg = cv2.imread(srcImgPath)
  srcImg = cv2.cvtColor(srcImg,cv2.COLOR_BGR2RGB)
  srcImg_gray = cv2.cvtColor(srcImg, cv2.COLOR_RGB2GRAY)

  destImg = cv2.imread(destImgPath)
  destImg = cv2.cvtColor(destImg,cv2.COLOR_BGR2RGB)
  destImg_gray = cv2.cvtColor(destImg, cv2.COLOR_RGB2GRAY)

  if showImages:
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, constrained_layout=False, figsize=(10,4))
    ax1.imshow(destImg, cmap="gray")
    ax1.set_xlabel("dest image", fontsize=14)

    ax2.imshow(srcImg, cmap="gray")
    ax2.set_xlabel("Src image (Image to be transformed)", fontsize=14)
    
    plt.show()

  return [srcImg, destImg]

In [11]:
def inputAndFormatFlowfile(flowPath):
  '''
  Input: flowPath
  flowpath: path to the .flo file between two images ->str
  Output: flow
  flow: the formated optical_flow -> ndArray of heightxbreathx2 of the image sizes 
  '''
  path = Path(flowPath) 
  with path.open(mode='r') as flo:
      np_flow = np.fromfile(flo, np.float32)
      # print(np_flow.shape)

  with path.open(mode='r') as flo:
    tag = np.fromfile(flo, np.float32, count=1)[0]
    width = np.fromfile(flo, np.int32, count=1)[0]
    height = np.fromfile(flo, np.int32, count=1)[0]

    # print('tag', tag, 'width', width, 'height', height)

    nbands = 2
    tmp = np.fromfile(flo, np.float32, count= nbands * width * height)
    flow = np.resize(tmp, (int(height), int(width), int(nbands)))
    return flow

In [12]:
def pointCorrespondenceFromOpticalFlow(flow, padding):
  '''
  Input: flow
  flow: the formatted optical flow file -> ndarray length,breadth,2
  padding: padding to add to destImg
  OutPut: [ptsA and ptsB] -> list
  ptsA: point correspondences in the destImg (original) -> ndarray length*breadth,2
  ptsB: point correspondences in the srcImg(new, been transformed) -> ndarray length*breadth,2
  '''
  ptsA = np.zeros(flow.shape)
  ptsB = np.zeros(flow.shape)

  for i in range(ptsA.shape[0]):
    for j in range(ptsA.shape[1]):
      ptsA[i,j] = np.array([i,j], dtype=np.float) + np.array([padding,padding])
      ptsB[i,j] = np.array([i,j])  +  (np.array(flow[j, i]))

  ptsA = np.reshape(ptsA, (-1, 2))
  ptsB = np.reshape(ptsB, (-1, 2))
  return [ptsA, ptsB]

In [13]:
def findTransformation(ptsA, ptsB, threshold=1):
  '''
    ptsA: point correspondences in the destImg (original) -> ndarray length*breadth,2
    ptsB: point correspondences in the srcImg(new, been transformed) -> ndarray length*breadth,2
    threshold: ransac threshold. 
    Output:
    H - homography
    status - mask of ransac accepted or rejected
  '''
  if transformation == "Homography":
    (H, status) = cv2.findHomography(ptsB, ptsA, cv2.RANSAC, threshold)
  elif transformation == "Affine":
    (H, status) = cv2.estimateAffine2D(ptsB, ptsA, method = cv2.RANSAC, ransacReprojThreshold = threshold)
  return [H, status]

In [14]:
def visualizeStitch(srcImg, destImg, H, padding, showImages=True):
  '''
  srcImg: 256x256 representation of srcImage ->ndarray
  destImg: 256x256 representation of destImage ->ndarray
  H - homography
  '''
  ht, wd, cc= destImg.shape

  ww = wd + (2*padding)
  hh = ht + (2*padding)

  xx = (ww - wd) // 2
  yy = (hh - ht) // 2

  if transformation == "Homography":
    result = cv2.warpPerspective(srcImg, H, (ww, hh))
  elif transformation == "Affine":
    result = cv2.warpAffine(srcImg, H, (ww, hh))

  result[yy:yy+ht, xx:xx+wd] = destImg

  if showImages:
    plt.figure(figsize=(10,4))
    plt.imshow(result)

    plt.show()

In [15]:
image_width = 256
def VisualizeOutliers(destImgPath, status, showImages):
  if showImages:
    ransac_fail = np.where(np.any(status==0, axis=1))[0]
    convert_to_pixel = lambda t: [t//image_width, t%image_width]
    outlier_pixels = np.array([convert_to_pixel(p) for p in ransac_fail])
    # print(outlier_pixels)
    destImg = Image.open(destImgPath)
    fig = plt.figure(figsize=(10,4))
    plt.xlabel("outliers",fontsize=14)
    if (outlier_pixels.size):
      plt.plot(outlier_pixels[:, 1],outlier_pixels[:, 0],'r.')
    plt.imshow(destImg)

In [16]:
def getHomographyAndStitch(srcImgPath, destImgPath, flowpath, padding, threshold, showImages = True):

  '''
  Inputs: srcImgPath, destImgPath 
  srcImgPath : path to the image been transformed, usually the new frame seen ->str
  destImgPath : path to the image that acts as destination(src image is trasformed to it), usually the old frame->str
  flowpath: path to the .flo file between two images ->str
  flow: the formatted optical flow file -> ndarray length,breadth,2
  padding: padding to add to destImg
  '''
  srcImg, destImg = inputAndVisualizeStitchPair(srcImgPath, destImgPath, showImages)
  flow = inputAndFormatFlowfile(flowPath)
  ptsA, ptsB =  pointCorrespondenceFromOpticalFlow(flow, padding)
  H, status = findTransformation(ptsA, ptsB, threshold)
  visualizeStitch(srcImg, destImg, H, padding, showImages)
  VisualizeOutliers(destImgPath, status, showImages)

  return H

#### Use functions to perform first stitch.

### Join 5 frames together using functions made

In [17]:
imgDirPath = '/content/cropped_' + video_name+ '_images'
from os import listdir
imgPaths =  [ imgDirPath + '/' + f  for f  in sorted(listdir(imgDirPath))]

In [18]:
flowFileDirPath = '/content/flow_files_'+video_name
flowPaths =  [ flowFileDirPath + '/' + f  for f  in sorted(listdir(flowFileDirPath))]

### Global registration

In [19]:
padding_size = 1000

In [20]:
def pointCorrespondenceWithoutPadding(flow):
  '''
  Input: flow
  flow: the formatted optical flow file -> ndarray length,breadth,2
  padding: padding to add to destImg
  OutPut: [ptsA and ptsB] -> list
  ptsA: point correspondences in the destImg (original) -> ndarray length*breadth,2
  ptsB: point correspondences in the srcImg(new, been transformed) -> ndarray length*breadth,2
  '''
  ptsA = np.zeros(flow.shape)
  ptsB = np.zeros(flow.shape)

  for i in range(ptsA.shape[0]):
    for j in range(ptsA.shape[1]):
      ptsA[i,j] = np.array([i,j], dtype=np.float)
      ptsB[i,j] = np.array([i,j])  +  (np.array(flow[j,i]))

  ptsA = np.reshape(ptsA, (-1, 2))
  ptsB = np.reshape(ptsB, (-1, 2))
  return [ptsA, ptsB]

In [21]:
def pointCorrespondenceWithPaddingForFirst(flow, padding= padding_size):
  '''
  Input: flow
  flow: the formatted optical flow file -> ndarray length,breadth,2
  padding: padding to add to destImg
  OutPut: [ptsA and ptsB] -> list
  ptsA: point correspondences in the destImg (original) -> ndarray length*breadth,2
  ptsB: point correspondences in the srcImg(new, been transformed) -> ndarray length*breadth,2
  '''
  ptsA = np.zeros(flow.shape)
  ptsB = np.zeros(flow.shape)

  for i in range(ptsA.shape[0]):
    for j in range(ptsA.shape[1]):
      ptsA[i,j] = np.array([i,j], dtype=np.float) + np.array([padding,padding])
      ptsB[i,j] = np.array([i,j])  +  (np.array(flow[j,i]))

  ptsA = np.reshape(ptsA, (-1, 2))
  ptsB = np.reshape(ptsB, (-1, 2))
  return [ptsA, ptsB]

In [22]:
def OldMethodOfVisualization(srcImg, destImg, H, showImages=True):
  width = destImg.shape[1] + srcImg.shape[1]
  height = destImg.shape[0] + srcImg.shape[0]

  if transformation == "Homography":
    result = cv2.warpPerspective(srcImg, H, (width, height))
  elif transformation == "Affine":    
    result = cv2.warpAffine(srcImg, H, (width, height))

  result[0:destImg.shape[0], 0:destImg.shape[1]] = destImg

  if showImages:
    plt.figure(figsize=(10,5))
    plt.imshow(result)

    plt.axis('off')
    plt.show()

In [23]:
padding = padding_size
threshold = 3
if transformation == "Homography":
  H_array = np.zeros((len(imgPaths)-1, 3,3 ))
elif transformation == "Affine":    
  H_array = np.zeros((len(imgPaths)-1, 2,3 ))
showImages = True

In [24]:
for i in range(len(imgPaths) - 1):
  destImgPath = imgPaths[i] #the previous image
  srcImgPath =  imgPaths[i + 1];
  flowPath = flowPaths[i]
  srcImg, destImg = inputAndVisualizeStitchPair(srcImgPath, destImgPath, showImages)
  flow = inputAndFormatFlowfile(flowPath)
  if i==0:
    ptsA, ptsB = pointCorrespondenceWithPaddingForFirst(flow, padding)
  else:
    ptsA, ptsB = pointCorrespondenceWithoutPadding(flow)
  H, status = findTransformation(ptsA, ptsB, threshold)
  if i==0:
    visualizeStitch(srcImg, destImg, H, padding, showImages)
  else:
    OldMethodOfVisualization(srcImg, destImg, H, showImages)
  VisualizeOutliers(destImgPath, status, showImages) 

  H_array[i] = H

Output hidden; open in https://colab.research.google.com to view.

Try and join them together, use alpha channel, to make them transparent a bit then overlap